In [1]:
#Importando as libs
import tweepy
import pandas as pd     
import numpy as np 



#import time
#import re
#import warnings
## Pacotes para visualização 
#import matplotlib.pyplot as plt
#from textblob import TextBlob as tb
##MAPA
#from geopy.geocoders import Nominatim
#import folium
#from folium import plugins
#from wordcloud import WordCloud, STOPWORDS

In [2]:
#Credenciais para utilização da API do Twitter

consumer_key = "ThjEtKexjpxXSEFdIskFAJIa1"
consumer_secret = "xVAOsFNAov5zBeToU6pQlTVGOGFY3yhaSdChzmAIq1LLFvNAs2"
access_token = "989683030229995521-lKpEaaN5RMMpJg99BIN0x4KFdEvJjde"
access_token_secret = "M9ibsxsupWqJ7Aby2FXtsPidLvrvmUIwTtNxDhXDDlO6Q"

In [3]:
#Realizar autenticação no Twitter

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
token = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True,retry_count=5)

In [4]:
# Definir listas de armazenamento
tweets = []
info = []

#Definir que palavra deseja pesquisar no Twitter
keyword = ("covid-19  OR  covid  OR  coronavirus OR pandemic OR coronga OR corona OR quarentena OR pandemia")

In [5]:
for tweet in tweepy.Cursor(token.search,
                           q=keyword, 
                           tweet_mode='extended',
                           rpp = 100,
                           result_type="mixed",
                           lang='pt', 
                           include_entities=True).items(10000): #serão coletados 10000 tweets
    if 'retweeted_status' in dir(tweet):
        aux=tweet.retweeted_status.full_text
    else:
        aux=tweet.full_text
        
    newtweet = aux.replace("\n", " ")
   
    tweets.append(newtweet)
    info.append(tweet)

Rate limit reached. Sleeping for: 695
Rate limit reached. Sleeping for: 694
Rate limit reached. Sleeping for: 694


In [6]:
# Para verificar a quantidade de tweets coletado use a função "len()"

print("Total de tweets coletados %s." % (len(info)))

Total de tweets coletados 10000.


In [7]:
tweets_df = pd.DataFrame(tweets, columns=['Tweets']) 

tweets_df['len']  = np.array([len(tweet) for tweet in tweets])
tweets_df['ID']   = np.array([tweet.id for tweet in info])
tweets_df['USER']   = np.array([tweet.user.screen_name for tweet in info])
tweets_df['userName'] = np.array([tweet.user.name for tweet in info])
tweets_df['User Location']    = np.array([tweet.user.location for tweet in info])
tweets_df['Language'] = np.array([tweet.user.lang for tweet in info])
tweets_df['Date'] = np.array([tweet.created_at for tweet in info])
tweets_df['Source'] = np.array([tweet.source for tweet in info])
tweets_df['Likes']  = np.array([tweet.favorite_count for tweet in info])
tweets_df['Retweets']    = np.array([tweet.retweet_count for tweet in info])     

tweets_df.to_csv("tweets_covid_2.csv")

In [8]:
tweets_df = pd.read_csv("tweets_covid_2.csv")
tweets_df.head(3)

,Unnamed: 0,Tweets,len,ID,USER,userName,User Location,Language,Date,Source,Likes,Retweets
0,0,Diretor responsável pelo Enem morre de Covid e...,103,1348773192853200899,folha,Folha de S.Paulo,"São Paulo, Brazil",NaN,2021-01-11 23:26:02,Folha com welcome-app 1.0,25363,3845
1,1,'Bolsonaro tentou sabotar medidas contra Covid...,123,1349332869869690881,g1,G1,Brasil,NaN,2021-01-13 12:30:00,TweetDeck,10140,947
2,2,O governo não vai adiar o Enem no meio de um n...,280,1348735575801651200,TatiNefertari,Tati Nefertari,NaN,NaN,2021-01-11 20:56:34,Twitter for Android,22863,4728


In [9]:
# Tweets com maior número de LIKES
likes_max = np.max(tweets_df['Likes']) #Função max do numpy identifica o valor máximo

likes = tweets_df[tweets_df.Likes == likes_max].index[0] #pega o primeiro tweet com valor máximo de curtidas

print("O tweet com mais curtidas (likes) é: \n{}".format(tweets_df['Tweets'][likes]))
print("Número de curtidas: {}".format(likes_max))

O tweet com mais curtidas (likes) é: 
Diretor responsável pelo Enem morre de Covid em meio a pressao para adiar exame https://t.co/oINm57SVRG
Número de curtidas: 25363


In [10]:
print(np.sum(tweets_df['Likes'] == likes_max)) #conta quantos tweets possuem o mesmo valor máximo de curtidas

1


In [11]:
retweet_max  = np.max(tweets_df['Retweets']) #retorna o valor máximo

retweet  = tweets_df[tweets_df.Retweets == retweet_max].index[0] #pega o primeiro tweet com valor máximo de Retweets

print("O tweet com mais retweet é: \n{}".format(tweets_df['Tweets'][retweet]))
print("Número de retweets: {}".format(retweet_max))

O tweet com mais retweet é: 
Aborto legalizado na Argentina: "lamento profundamente pela vida das crianças"  190 mil mortos pela COVID-19 no Brasil: "não sou coveiro" https://t.co/SaLVKhNucl
Número de retweets: 31075


In [12]:
print(np.sum(tweets_df['Retweets'] == retweet_max)) #conta quantos tweets possuem o mesmo valor máximo de Retweets

1


In [13]:
stopwords= set(STOPWORDS)

new_words = []
with open("stopwords.txt", 'r') as f:
    [new_words.append(word) for line in f for word in line.split()]

new_stopwords = stopwords.union(new_words)

words = ' '.join(tweets_df['Tweets'])

words_clean = " ".join([word for word in words.split()
                            if 'https' not in word
                                and not word.startswith('@')
                                and word != 'RT'
                            ])

warnings.simplefilter('ignore')

wc = WordCloud(min_font_size=20, 
               max_font_size=300, 
               background_color='white', 
               mode="RGB",
               stopwords=new_stopwords,
               width=5000, 
               height=2000,
               normalize_plurals= True).generate(words_clean)

plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.savefig('covid_cloud_pt.png', dpi=300)
plt.show()

NameError: name 'STOPWORDS' is not defined